In [ ]:
import csv, os, shutil, datetime, sys, pyodbc, decimal, subprocess, re, json
from pathlib import Path

In [ ]:
import logging
logfile = Path('C:/Users/***/Archive/Logs/Logfile.txt') # set to variable later
logging.basicConfig(filename=str(logfile),
                    filemode='a',
                    format='%(asctime)s,%(msecs)03d %(name)s %(levelname)s %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.DEBUG, force=True)

In [ ]:
logging.info("Running Main")

In [ ]:
current_dateTime = datetime.datetime.today().strftime('%Y-%m-%d') # actual datetime_ to be added into filename for archiving
folder = Path('C:/Users/***/csv') # set to variable later
Path(f'{folder}/Archive').mkdir(parents=True, exist_ok=True)
folder_arch = Path('C:/Users/***/csv/Archive/') #set to variable later
folder_SQL_Scripts = Path('C:/Users/***/SQL_Scripts/') #set to variable later
sql_files = list(folder_SQL_Scripts.glob('*_load_.sql')) # adjust for prod
 # indicator for the load type - monthly or weekly
load_type = ''
weekly_type = ['WEEKLY.csv','jti_price_surowe.csv']
monthly_type = ['MONTHLY.csv']
#ALL LOAD TYPE? M+W? - add later
sql_file_w_DQ = list(folder_SQL_Scripts.glob('*.sql'))
sqlQuery_w_DQ = ''
sqlQuery_w_ref = ''
sqlQuery_w_pr_ref = ''
sqlQuery_w_p_DQ = ''
sqlQuery_m_DQ = ''
sqlQuery_m_ref = ''


In [ ]:
logging.info("Paths set successfully")

In [ ]:
for f in sql_file_w_DQ:
    sql_path = f
    sql_f = os.path.basename(f)
    for line in open(f,'r',encoding='utf-16'):
        logging.debug(line)
        if 'wweekly_load_prices_DQ' in str(f):
            sqlQuery_w_p_DQ = line
        elif 'wweekly_load_ref_DQ' in str(f):
            sqlQuery_w_ref = line
        elif 'wweekly_load_vol_DQ' in str(f):
            sqlQuery_w_DQ = line
        elif 'wweekly_load_prices_ref_DQ' in str(f):
            sqlQuery_w_pr_ref = line
        elif 'mmonthly_load_ref_DQ' in str(f):
            sqlQuery_m_ref = line
        elif 'mmonthly_load_vol_DQ' in str(f):
            sqlQuery_m_DQ = line

In [ ]:
# data reference connection as well as final destination for data load (master_new_temp)


with open('C:/Users/***/SQL_conn_settings.json', 'r') as file:
    data = json.load(file)

conn_0 = pyodbc.connect(f"Driver={data['Driver']};" 
                      f"Server={data['Server']};"
                      f"Database={data['Database']};"
                      f"Trusted_Connection={data['Trusted_Connection']};")


In [ ]:
CSV_file = list(folder.glob('*.csv')) # very important thing - if .glob used without list - it will contain a path value untill first use, after` that it is erased - this caused my archiving to break
folder_arch_A = folder_arch.glob('*.csv')
logging.debug(CSV_file) # debug line

In [ ]:
CSV_file_name = []
for f in CSV_file:
    CSV_file_name.append(os.path.basename(f))
#load_type indication
if set(weekly_type).issubset(CSV_file_name):
    load_type = 'W'
    print('Weekly load type detected')
    logging.info('Weekly load type detected')
    for k in list(folder_SQL_Scripts.glob('Weekly_load.sql')):
        sql_p = k
        sql_f = os.path.basename(k)
elif set(monthly_type).issubset(CSV_file_name):
    print('Monthly load type detected')
    logging.info('Monthly load type detected')
    load_type = 'M'
    for k in list(folder_SQL_Scripts.glob('Monthly_load.sql')):
        sql_p = k
        sql_f = os.path.basename(k)
else:
    print(f'No csv file found in {folder}, loadtype is unknown')

In [ ]:
# executing powershell loader with subprocess library 
ps_script = r"C:\Users\***\csv\SQL.ps1" # change to relative path later
function = load_type

# params add here
cmd = [
    "powershell.exe",
    "-ExecutionPolicy", "Bypass",
    "-Command",
    f'. "{ps_script}"; {function}'

]

# run
result = subprocess.run(cmd, capture_output=True, text=True)

# read shell reply
logging.debug(result.stdout)
logging.debug(result.stderr)
print("Exit code:", result.returncode)


In [ ]:
def DQ_check(conn, cur): # DQ query - have to adjust for every concrete csv ingested
#CHECKING NUMBER OF ROWS AND SUM OF VOL AND COMPARING AGAINST CLEAN DATA - DIFFERENCES MORE THAN 25% WILL RAISE ALERT#
    if 'W' == load_type:
        cur_0.execute(sqlQuery_w_ref)
        K = cur_0.fetchall()
        
        for x,v in K: # Getting average meanings for num of rows and gross vol from benchmark DB
            logging.info(f'Average Gross Volume: {x} Average Number of rows: {v} - WEEKLY')
            print(f'Average Gross Volume: {x} Average Number of rows: {v}')
            #None - remove hash after debug 
    
            
        cur_0.execute(sqlQuery_w_DQ)
        L = cur_0.fetchall()
        for k,c,i in L:
            logging.info(f'Gross Volume ({k}) ; average meaning ({x}) ; date  {i}- WEEKLY')
    
            if ((decimal.Decimal(k)/x)-1)*100 >= 25:
                print(f'Gross Volume ({k}) is higher than average meaning ({x}) for more than 25%, please review date from {i}')
                print('Do you want to continue loading? y/n')
                while True:
                    answer = input()
                    if answer == 'y':
                        return(answer)
                        break
                    if answer == 'n':
                        return(answer)
                        break
                    else:
                        print('Please provide correct answer, y/n')
    
            elif ((float(c)/v)-1)*100 >= 25:
                print(f'Number of rows ({k}) is higher than average meaning ({v}) for more than 25%, please review date from {i}')
                print('Do you want to continue loading? y/n')
                while True:
                    answer = input()
                    if answer == 'y':
                        return(answer)
                        break
                    if answer == 'n':
                        return(answer)
                        break
                    else:
                        print('Please provide correct answer, y/n')
    
            elif ((decimal.Decimal(k)/x)-1)*100 <= -25:
                print(f'Gross Volume ({k}) is lower than average meaning ({x}) for more than 25%, please review date from {i}')
                print('Do you want to continue loading? y/n')
                while True:
                    answer = input()
                    if answer == 'y':
                        return(answer)
                        break
                    if answer == 'n':
                        return(answer)
                        break
                    else:
                        print('Please provide correct answer, y/n')
    
            elif ((float(c)/v)-1)*100 <= -25:
                print(f'Number of rows ({k}) is lower than average meaning ({v}) for more than 25%, please review date from {i}')
                print('Do you want to continue loading? y/n')
                while True:
                    answer = input()
                    if answer == 'y':
                        return(answer)
                        break
                    if answer == 'n':
                        return(answer)
                        break
                    else:
                        print('Please provide correct answer, y/n')
    
            else:
                print(f'Date: {i} Gross Volume: {k} Number of rows: {c}')
    
    #---------------- checking prices DQ
        print('---------------------- Prices file analysis ----------------------')
        logging.info('---------------------- Prices file analysis ----------------------')
        cur_0.execute(sqlQuery_w_pr_ref)
        P = cur_0.fetchall()
        
        for pp in P: # Getting average meanings for num of rows from prices table
            po = int(pp[0]) # BECAUSE I WAS GETTING CONN.OBJECT INSTEAD OF NUMERIC VALUE - NO CALCULATION COULD BE APPLIED IN DQ STAGE
            print(f'Average Number of rows per week: {po}')
            logging.info(f'Average Number of rows per week: {po}')
            
    
    
        cur_0.execute(sqlQuery_w_p_DQ)
        L = cur_0.fetchall()
        for no_lines,dt in L:
            logging.info(f'number of lines - {no_lines}; date - {dt}')
    
            if ((no_lines/po)-1)*100 >= 25:
                print(f'Number of rows ({no_lines}) is higher than average meaning ({pp}) for more than 25%, please review date from {dt}')
                print('Do you want to continue loading? y/n')
                while True:
                    answer = input()
                    if answer == 'y':
                        return(answer)
                        break
                    if answer == 'n':
                        return(answer)
                        break
                    else:
                        print('Please provide correct answer, y/n')
    
            elif ((no_lines/po)-1)*100 <= -25:
                print(f'Number of rows ({no_lines}) is lower than average meaning ({pp}) for more than 25%, please review date from {dt}')
                print('Do you want to continue loading? y/n')
                while True:
                    answer = input()
                    if answer == 'y':
                        return(answer)
                        break
                    if answer == 'n':
                        return(answer)
                        break
                    else:
                        print('Please provide correct answer, y/n')
    
            else:
                print(f'Date: {dt} Number of rows: {no_lines}')
    
    
    
    
    
    
            
        print('Do you want to continue loading? y/n')
        while True:
            answer = input()
            if answer == 'y':
                return(answer)
                break
            if answer == 'n':
                return(answer)
                break
            else:
                print('Please provide correct answer, y/n')
                
    elif 'M' == load_type:



        cur_0.execute(sqlQuery_m_ref)
        K = cur_0.fetchall()
        
        for x,v in K: # Getting average meanings for num of rows and gross vol from benchmark DB
            print(f'Average Gross Volume: {x} Average Number of rows: {v} - MONTHLY')
            logging.info(f'Average Gross Volume: {x} Average Number of rows: {v}')
             
    
            
        cur_0.execute(sqlQuery_m_DQ)
        L = cur_0.fetchall()
        for k,c,i in L:
            logging.info(f'Gross Volume ({k}) ; average meaning ({x}) ; date  {i}- MONTHLY')
    
            if ((decimal.Decimal(k)/x)-1)*100 >= 25:
                print(f'Gross Volume ({k}) is higher than average meaning ({x}) for more than 25%, please review date from {i}')
                print('Do you want to continue loading? y/n')
                while True:
                    answer = input()
                    if answer == 'y':
                        return(answer)
                        break
                    if answer == 'n':
                        return(answer)
                        break
                    else:
                        print('Please provide correct answer, y/n')
    
            elif ((float(c)/v)-1)*100 >= 25:
                print(f'Number of rows ({k}) is higher than average meaning ({v}) for more than 25%, please review date from {i}')
                print('Do you want to continue loading? y/n')
                while True:
                    answer = input()
                    if answer == 'y':
                        return(answer)
                        break
                    if answer == 'n':
                        return(answer)
                        break
                    else:
                        print('Please provide correct answer, y/n')
    
            elif ((decimal.Decimal(k)/x)-1)*100 <= -25:
                print(f'Gross Volume ({k}) is lower than average meaning ({x}) for more than 25%, please review date from {i}')
                print('Do you want to continue loading? y/n')
                while True:
                    answer = input()
                    if answer == 'y':
                        return(answer)
                        break
                    if answer == 'n':
                        return(answer)
                        break
                    else:
                        print('Please provide correct answer, y/n')
    
            elif ((float(c)/v)-1)*100 <= -25:
                print(f'Number of rows ({k}) is lower than average meaning ({v}) for more than 25%, please review date from {i}')
                print('Do you want to continue loading? y/n')
                while True:
                    answer = input()
                    if answer == 'y':
                        return(answer)
                        break
                    if answer == 'n':
                        return(answer)
                        break
                    else:
                        print('Please provide correct answer, y/n')
    
            else:
                print(f'Date: {i} Gross Volume: {k} Number of rows: {c}')
        

        print('Do you want to continue loading? y/n')
        while True:
            answer = input()
            if answer == 'y':
                return(answer)
                break
            if answer == 'n':
                return(answer)
                break
            else:
                print('Please provide correct answer, y/n')


In [ ]:
def exec_sql_file(cur_0, sql_p):
    #SCRIPT TO RUN SQL FILE 
    #print ("\n[INFO] Executing SQL script file: '%s'" % (sql_p))
    statement = ""
    

    
    for line in open(sql_p,'r',encoding='utf-16'):
        #logging.info(f'SQL LINE RAN {line}')
        #print(line)
        
        if re.match(r'--', line):  # ignore sql comment lines
            continue
        if not re.search(r';$', line):  # keep appending lines that don't end in ';'
            statement = statement + line
            
        else:  # when you get a line ending in ';' then exec statement and reset for next statement
            statement = statement + line
            
            print ("\n\n[DEBUG] Executing SQL statement:\n%s" % (statement))
            #try:
            with conn_0:
                cur_0 = conn_0.cursor()
                cur_0.execute(statement)
                conn_0.commit()
            #except (OperationalError, ProgrammingError) as e:
                #print("\n[WARN] MySQLError during execute statement \n\tArgs: '%s'" % (str(e.args)))

            statement = ""
    

In [ ]:
with conn_0:
    cur_0 = conn_0.cursor()
    repl = DQ_check(conn_0, cur_0)

In [ ]:
if repl == 'n':
    None
    try:
        sys.exit()
    except:
        print('Exiting')
        logging.info('Exiting, answer is NO')
elif repl == 'y':
    with conn_0:
        cur_0 = conn_0.cursor()
        print('Loading started')
        cur_0.execute("INSERT INTO [M].[dbo].[new] SELECT * FROM [M].[dbo].[temp_temp]")
        cur_0.execute('DELETE from [M].[dbo].[temp_temp]')
        cur_0.execute("INSERT INTO [M].[dbo].[prices_new] SELECT * FROM [M].[dbo].[temp_prices]")
        cur_0.execute('DELETE from [M].[dbo].[temp_prices]')
        
    
        print('Data loaded, archiving files')
        logging.info('SQL INSERTS into temp and temp_prices as well as deletes of temp and temp_prices done')
        
    for F in list(CSV_file): #renaming in the archived folder
        CSV_file_name = os.path.basename(F)
        CSV_file_name_F = str(CSV_file_name[:-4])
        archiveName = str(CSV_file_name_F)+'_'+ str(current_dateTime+'.csv')
        shutil.move(folder_arch/F,folder_arch/archiveName)
            
    print('Archivisation complete')
    logging.info(f'Archivisation{folder_arch/archiveName}')






In [ ]:
exec_sql_file(cur_0,sql_p)

cur_0.close()
conn_0.close()